In [30]:
## capcity

import pandas as pd
import numpy as np

# 读取Excel文件
df = pd.read_excel('X_fixed.xlsx')

# 将DataFrame对象转换为Numpy数组
numpy_array = df.values

# 打印Numpy数组
# print(numpy_array.shape)
X_train = numpy_array
# X_test = numpy_array[51:]
df2 = pd.read_excel('Y_fixed_retion.xlsx')

# 将DataFrame对象转换为Numpy数组
numpy_array2 = df2.values
Y_train = numpy_array2
# Y_test = numpy_array2[51:,:]
# print(numpy_array2.shape)
numpy_array2.shape

(60, 11)

In [17]:
np.save('XX_55.npy',X_train)
np.save('XX_55_test.npy',X_test)
np.save('YY_55.npy',y_train)
np.save('YY_55_test.npy',y_test)

In [32]:
mean_list=[]
std_list=[]
for i in range(numpy_array.shape[1]):
    mean_list.append(numpy_array[:,i].mean())
    std_list.append(numpy_array[:,i].std())
np.save('mean_list_X.npy',mean_list)
np.save('std_list_X.npy',std_list)

mean_listY=[]
std_listY=[]
for i in range(numpy_array2.shape[1]):
    mean_listY.append(numpy_array2[:,i].mean())
    std_listY.append(numpy_array2[:,i].std())
np.save('mean_list.npy',mean_listY)
np.save('std_list.npy',std_listY)


In [33]:
import numpy as np
import pandas as pd
extra_dixt=np.load('temp_.npy')
X_basis=np.load('X_basis.npy').squeeze()
df3 = pd.read_excel('dict1.xlsx')
# 将DataFrame对象转换为Numpy数组
dict1 = df3.values
# dict1.shape
# mapping={11:[0.0537,0.0076,2.87825,14.22066, 102.02500,0.04359,1.93946,18.7021],
#          5:}
mapping={}
for iidx in range(dict1.shape[0]):
    # str_1=str(dict1[iidx,0])
    mapping[dict1[iidx,0]]=dict1[iidx,1:]
for jjdx in range(extra_dixt.shape[0]):
    mapping[X_basis[jjdx]]=extra_dixt[jjdx]




In [34]:
## 插值基向量

for i in range(1,84):
    for j in range(60):
        a=np.array([i,j*1e-3])
        # print(np.array(mapping[i]))
        a=np.concatenate((a,np.array(mapping[i])),axis=0)
        if i==1 and j==0:
            X_basis=a
        else:
            X_basis = np.vstack((X_basis, a))
# for i in range(X_basis.shape[0]):
#     print(X_basis[i])
print(X_basis.shape)


(4980, 9)


In [35]:
## 高斯过程回归
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.preprocessing import StandardScaler

import numpy as np
from sklearn.metrics import mean_squared_error
def calculate_rmse(y_true, y_pred):
    """
    计算两个变量的RMSE
    
    参数:
    y_true -- 真实值，形状为(n_samples,)或(n_samples, 1)
    y_pred -- 预测值，形状为(n_samples,)或(n_samples, 1)
    
    返回:
    rmse -- 均方根误差
    """
    # 确保输入是一维数组
    y_true = np.array(y_true).flatten()
    y_pred = np.array(y_pred).flatten()
    
    # 检查长度是否一致
    if len(y_true) != len(y_pred):
        raise ValueError("真实值和预测值的长度必须相同")
    
    # 计算MSE然后取平方根得到RMSE
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    
    return rmse

class MultiOutputGPR:
    def __init__(self, kernel=None, n_restarts_optimizer=10, random_state=None):
        self.kernel = kernel
        self.n_restarts_optimizer = n_restarts_optimizer
        self.random_state = random_state
        self.models = []
        self.X_scaler = StandardScaler()
        self.Y_scalers = []
        
    def fit(self, X, Y):
        # 标准化输入
        X_scaled = self.X_scaler.fit_transform(X)
        
        # 为每个输出维度训练一个GPR模型
        n_outputs = Y.shape[1]
        self.models = []
        self.Y_scalers = []
        
        for i in range(n_outputs):
            # 标准化当前输出维度
            y_scaler = StandardScaler()
            y_scaled = y_scaler.fit_transform(Y[:, i].reshape(-1, 1)).flatten()
            self.Y_scalers.append(y_scaler)
            
            # 创建并训练GPR模型
            if self.kernel is None:
                kernel = C(1.0, (1e-3, 1e3)) * RBF([1.0] * X.shape[1], (1e-2, 1e2))
            else:
                kernel = self.kernel
                
            gpr = GaussianProcessRegressor(
                kernel=kernel,
                n_restarts_optimizer=self.n_restarts_optimizer,
                random_state=self.random_state
            )
            
            gpr.fit(X_scaled, y_scaled)
            self.models.append(gpr)
            
        return self
    
    def predict(self, X, return_std=False):
        X_scaled = self.X_scaler.transform(X)
        n_outputs = len(self.models)
        
        if return_std:
            y_pred = np.zeros((X.shape[0], n_outputs))
            y_std = np.zeros((X.shape[0], n_outputs))
            
            for i, (model, scaler) in enumerate(zip(self.models, self.Y_scalers)):
                pred, std = model.predict(X_scaled, return_std=True)
                y_pred[:, i] = scaler.inverse_transform(pred.reshape(-1, 1)).flatten()
                y_std[:, i] = std * scaler.scale_  # 调整标准差
                
            return y_pred, y_std
        else:
            y_pred = np.zeros((X.shape[0], n_outputs))
            
            for i, (model, scaler) in enumerate(zip(self.models, self.Y_scalers)):
                pred = model.predict(X_scaled)
                y_pred[:, i] = scaler.inverse_transform(pred.reshape(-1, 1)).flatten()
                
            return y_pred

# 使用自定义类
np.random.seed(42)

# 输入数据 (4, 2)
# X = np.random.randn(4, 2)
X=X_train
X.shape
# 输出数据 (4, 21)
Y = Y_train #np.random.randn(4, 21)

# 创建并训练模型
multi_gpr = MultiOutputGPR(n_restarts_optimizer=10, random_state=42)
multi_gpr.fit(X, Y)
        
Y_pred_gau, Y_std = multi_gpr.predict(X_basis, return_std=True)
print(Y_pred_gau.shape)

/home/gnn/miniconda3/envs/meshcnn/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:455: ConvergenceWarning: The optimal value found for dimension 2 of parameter k2__length_scale is close to the specified upper bound 100.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/gnn/miniconda3/envs/meshcnn/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:455: ConvergenceWarning: The optimal value found for dimension 3 of parameter k2__length_scale is close to the specified upper bound 100.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/gnn/miniconda3/envs/meshcnn/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:455: ConvergenceWarning: The optimal value found for dimension 4 of parameter k2__length_scale is close to the specified upper bound 100.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/gnn/miniconda3/envs/me

(4980, 11)


In [36]:
## 随机森林
from sklearn.ensemble import RandomForestClassifier  # 分类
from sklearn.ensemble import RandomForestRegressor   # 回归
import pandas as pd
import numpy as np

model = RandomForestRegressor(n_estimators=100, random_state=42)   # 对于回归

import numpy as np
from sklearn.metrics import mean_squared_error
def calculate_rmse(y_true, y_pred):
    """
    计算两个变量的RMSE
    
    参数:
    y_true -- 真实值，形状为(n_samples,)或(n_samples, 1)
    y_pred -- 预测值，形状为(n_samples,)或(n_samples, 1)
    
    返回:
    rmse -- 均方根误差
    """
    # 确保输入是一维数组
    y_true = np.array(y_true).flatten()
    y_pred = np.array(y_pred).flatten()
    
    # 检查长度是否一致
    if len(y_true) != len(y_pred):
        raise ValueError("真实值和预测值的长度必须相同")
    
    # 计算MSE然后取平方根得到RMSE
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    
    return rmse

model.fit(X_train, Y_train)  # X_train 是训练特征，y_train 是训练标签
Y_pred_ran = model.predict(X_basis)  # X_test 是测试特征
print(Y_pred_ran.shape)

(4980, 11)


In [ ]:
Y_temp = (Y_pred_ran+Y_pred_gau)/2
#X_basis.shape=[4980,9]nv
np.save('x_interplot.npy',X_basis)
timess=0
for iidx in range(X_basis.shape[0]):
    for jjjdx in range(X_train.shape[0]):
        if X_basis[iidx,0]==X_train[jjjdx,0] and X_basis[iidx,1]==X_train[jjjdx,1]:
            timess+=1
            # print('执行替换')
            Y_temp[iidx]=Y_train[jjjdx]
            if X_train[jjjdx,0]==51:
                print(Y_train[jjjdx,-2])
print(timess)
np.save('y_interplot.npy',Y_temp)

197.7
192.2
159.8
60


In [38]:
## 均值方差
## capcity
mean_list_inter=[]
std_list_inter=[]
for ifd in range(X_basis.shape[1]):
    mean_list_inter.append(X_basis[:,ifd].mean())
    std_list_inter.append(X_basis[:,ifd].std())
np.save('mean_list_inter_X.npy',mean_list_inter)
np.save('std_list_inter_X.npy',std_list_inter)

mean_list_inter_Y=[]
std_list_inter_Y=[]
for ifd in range(Y_temp.shape[1]):
    mean_list_inter_Y.append(Y_temp[:,ifd].mean())
    std_list_inter_Y.append(Y_temp[:,ifd].std())
np.save('mean_list_inter.npy',mean_list_inter_Y)
np.save('std_list_inter.npy',std_list_inter_Y)

In [ ]:
## retent
